In [0]:

# Configurações de conexão
jdbcHostname = "psql-mock-database-cloud.postgres.database.azure.com"
jdbcPort = 5432
jdbcDatabase = "ecom1692155331663giqokzaqmuqlogbu"
jdbcUrl = f"jdbc:postgresql://{jdbcHostname}:{jdbcPort}/{jdbcDatabase}"

# Propriedades de conexão
connectionProperties = {
  "user" : "eolowynayhvayxbhluzaqxfp@psql-mock-database-cloud",
  "password" : "hdzvzutlssuozdonhflhwyjm",
  "driver" : "org.postgresql.Driver"
}

In [0]:
# Usar uma consulta SQL para listar as tabelas no banco de dados
query = "(SELECT table_name FROM information_schema.tables WHERE table_schema = 'public') AS tabela_listagem"

# Ler o resultado da query no Spark DataFrame
df_tables = spark.read.jdbc(url=jdbcUrl, table=query, properties=connectionProperties)

# Mostrar as tabelas encontradas
df_tables.show(truncate=False)



+------------------+
|table_name        |
+------------------+
|customers         |
|employees         |
|offices           |
|orderdetails      |
|orders            |
|payments          |
|product_lines     |
|products          |
|pg_stat_statements|
|pg_buffercache    |
+------------------+



In [0]:
# Lista de tabelas específicas que você deseja copiar
tables_to_copy = [
    "customers", 
    "employees", 
    "offices", 
    "orderdetails", 
    "orders", 
    "payments", 
    "product_lines", 
    "products", 
    "pg_stat_statements"
]

# Iterar sobre cada tabela e realizar a ingestão
for table in tables_to_copy:
    print(f"Iniciando ingestão da tabela: {table}")
    
    # Ler cada tabela no DataFrame Spark
    df = spark.read.jdbc(url=jdbcUrl, table=f"public.{table}", properties=connectionProperties)
    
    # Definir o caminho onde os arquivos .parquet serão salvos
    output_path = f"/mnt/data/ecommerce/{table}.parquet"
    
    # Salvar os dados em formato parquet (1 arquivo por tabela)
    df.write.mode("overwrite").parquet(output_path)
    
    print(f"Tabela {table} foi salva como .parquet em {output_path}")

print("Ingestão completa!")

Iniciando ingestão da tabela: customers
Tabela customers foi salva como .parquet em /mnt/data/ecommerce/customers.parquet
Iniciando ingestão da tabela: employees
Tabela employees foi salva como .parquet em /mnt/data/ecommerce/employees.parquet
Iniciando ingestão da tabela: offices
Tabela offices foi salva como .parquet em /mnt/data/ecommerce/offices.parquet
Iniciando ingestão da tabela: orderdetails
Tabela orderdetails foi salva como .parquet em /mnt/data/ecommerce/orderdetails.parquet
Iniciando ingestão da tabela: orders
Tabela orders foi salva como .parquet em /mnt/data/ecommerce/orders.parquet
Iniciando ingestão da tabela: payments
Tabela payments foi salva como .parquet em /mnt/data/ecommerce/payments.parquet
Iniciando ingestão da tabela: product_lines
Tabela product_lines foi salva como .parquet em /mnt/data/ecommerce/product_lines.parquet
Iniciando ingestão da tabela: products
Tabela products foi salva como .parquet em /mnt/data/ecommerce/products.parquet
Iniciando ingestão da ta

In [0]:
# Definir o caminho base onde as tabelas Parquet estão armazenadas
base_path = "/mnt/data/ecommerce/"

# Lista das tabelas para serem lidas
tables = [
    "customers", 
    "employees", 
    "offices", 
    "orderdetails", 
    "orders", 
    "payments", 
    "product_lines", 
    "products", 
    "pg_stat_statements"
]

# Dicionário para armazenar os DataFrames
dataframes = {}

# Iterar sobre cada tabela, ler o arquivo Parquet e armazenar no dicionário
for table in tables:
    df = spark.read.parquet(f"{base_path}{table}.parquet")
    dataframes[table] = df  # Armazenar o DataFrame no dicionário com o nome da tabela
    print(f"Exibindo conteúdo da tabela: {table}")
    df.show(1)  # Exibir as primeiras linhas da tabela

Exibindo conteúdo da tabela: customers
+---------------+-------------+-----------------+------------------+----------+--------------+-------------+------+--------+-----------+-------+-------------------------+------------+
|customer_number|customer_name|contact_last_name|contact_first_name|     phone| address_line1|address_line2|  city|   state|postal_code|country|sales_rep_employee_number|credit_limit|
+---------------+-------------+-----------------+------------------+----------+--------------+-------------+------+--------+-----------+-------+-------------------------+------------+
|            103|         Jake|             King|           Carine |40.32.2555|54, rue Royale|         null|Nantes|Victoria|      44000| France|                     1370|    21000.00|
+---------------+-------------+-----------------+------------------+----------+--------------+-------------+------+--------+-----------+-------+-------------------------+------------+
only showing top 1 row

Exibindo conteúdo

## Query para encontrar o país com mais itens cancelados:

In [0]:
# Carregar as tabelas de interesse
orders_df = dataframes["orders"]
customers_df = dataframes["customers"]

# Registrar as tabelas como vistas temporárias para usar SQL
orders_df.createOrReplaceTempView("orders")
customers_df.createOrReplaceTempView("customers")

# Query para encontrar o país com mais itens cancelados
canceled_items_df = spark.sql("""
    SELECT c.country, COUNT(o.order_number) AS cancelled_items
    FROM orders o
    JOIN customers c ON o.customer_number = c.customer_number
    WHERE o.status = 'Cancelled'
    GROUP BY c.country
    ORDER BY cancelled_items DESC
    LIMIT 1
""")

# Mostrar o resultado da consulta
canceled_items_df.show()

# Salvar o resultado em uma tabela Delta
canceled_items_df.write.format("delta").mode("overwrite").save("/mnt/delta/canceled_items")



+-----------+---------------+
|    country|cancelled_items|
+-----------+---------------+
|New Zealand|              2|
+-----------+---------------+



### Query para calcular o faturamento da linha de produto mais vendida em 2005 com status 'Shipped'

In [0]:
# Carregar as tabelas de interesse
orders_df = dataframes["orders"]
orderdetails_df = dataframes["orderdetails"]
products_df = dataframes["products"]
product_lines_df = dataframes["product_lines"]

# Registrar as tabelas como vistas temporárias para usar SQL
orders_df.createOrReplaceTempView("orders")
orderdetails_df.createOrReplaceTempView("orderdetails")
products_df.createOrReplaceTempView("products")
product_lines_df.createOrReplaceTempView("product_lines")

# Query para calcular o faturamento da linha de produto mais vendida em 2005 com status 'Shipped'
revenue_df = spark.sql("""
    SELECT pl.product_line, SUM(od.quantity_ordered * od.price_each) AS total_revenue
    FROM orders o
    JOIN orderdetails od ON o.order_number = od.order_number
    JOIN products p ON od.product_code = p.product_code
    JOIN product_lines pl ON p.product_line = pl.product_line
    WHERE o.status = 'Shipped'
    AND YEAR(o.order_date) = 2005
    GROUP BY pl.product_line
    ORDER BY total_revenue DESC
    LIMIT 1
""")

# Mostrar o resultado da consulta
revenue_df.show()

# Salvar o resultado em uma tabela Delta
revenue_df.write.format("delta").mode("overwrite").save("/mnt/delta/revenue_2005")

+------------+-------------+
|product_line|total_revenue|
+------------+-------------+
|Classic Cars|    603666.99|
+------------+-------------+



In [0]:
orders_df.printSchema()
orderdetails_df.printSchema()
products_df.printSchema()
product_lines_df.printSchema()

root
 |-- order_number: integer (nullable = true)
 |-- order_date: date (nullable = true)
 |-- required_date: date (nullable = true)
 |-- shipped_date: date (nullable = true)
 |-- status: string (nullable = true)
 |-- comments: string (nullable = true)
 |-- customer_number: integer (nullable = true)

root
 |-- order_number: integer (nullable = true)
 |-- product_code: string (nullable = true)
 |-- quantity_ordered: integer (nullable = true)
 |-- price_each: decimal(10,2) (nullable = true)
 |-- order_line_number: short (nullable = true)

root
 |-- product_code: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- product_line: string (nullable = true)
 |-- product_scale: string (nullable = true)
 |-- product_vendor: string (nullable = true)
 |-- product_description: string (nullable = true)
 |-- quantity_in_stock: short (nullable = true)
 |-- buy_price: decimal(10,2) (nullable = true)
 |-- msrp: decimal(10,2) (nullable = true)

root
 |-- product_line: string (nullab

## Query para trazer nome, sobrenome e e-mail mascarado dos vendedores no Japão

In [0]:
# Carregar as tabelas de interesse
employees_df = dataframes["employees"]
offices_df = dataframes["offices"]

# Registrar as tabelas como vistas temporárias para usar SQL
employees_df.createOrReplaceTempView("employees")
offices_df.createOrReplaceTempView("offices")

# Query para trazer nome, sobrenome e e-mail mascarado dos vendedores no Japão
masked_email_df = spark.sql("""
    SELECT e.first_name, e.last_name, 
           CONCAT('***', SUBSTRING(e.email, POSITION('@' IN e.email), LENGTH(e.email))) AS masked_email
    FROM employees e
    JOIN offices o ON e.office_code = o.office_code
    WHERE o.country = 'Japan'
""")

# Mostrar o resultado da consulta
masked_email_df.show()

# Salvar o resultado em uma tabela Delta
masked_email_df.write.format("delta").mode("overwrite").save("/mnt/delta/masked_emails_japan")

+----------+---------+--------------------+
|first_name|last_name|        masked_email|
+----------+---------+--------------------+
|      Mami|    Nishi|***@classicmodelc...|
|   Yoshimi|     Kato|***@classicmodelc...|
+----------+---------+--------------------+



In [0]:
employees_df.printSchema()
offices_df.printSchema()

root
 |-- employee_number: integer (nullable = true)
 |-- last_name: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- extension: string (nullable = true)
 |-- email: string (nullable = true)
 |-- office_code: string (nullable = true)
 |-- reports_to: integer (nullable = true)
 |-- job_Title: string (nullable = true)

root
 |-- office_code: string (nullable = true)
 |-- city: string (nullable = true)
 |-- phone: string (nullable = true)
 |-- address_line1: string (nullable = true)
 |-- address_line2: string (nullable = true)
 |-- state: string (nullable = true)
 |-- country: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- territory: string (nullable = true)



In [0]:
# Carregar as tabelas de interesse
orders_df = dataframes["orders"]
customers_df = dataframes["customers"]

# Registrar as tabelas como vistas temporárias para usar SQL
orders_df.createOrReplaceTempView("orders")
customers_df.createOrReplaceTempView("customers")

# Query para encontrar o país com mais itens cancelados
canceled_items_df = spark.sql("""
    SELECT c.country, COUNT(o.order_number) AS cancelled_items
    FROM orders o
    JOIN customers c ON o.customer_number = c.customer_number
    WHERE o.status = 'Cancelled'
    GROUP BY c.country
    ORDER BY cancelled_items DESC
    LIMIT 1
""")

# Mostrar o resultado da consulta
canceled_items_df.show()

# Salvar o resultado em uma tabela Delta
canceled_items_df.write.format("delta").mode("overwrite").save("/mnt/delta/canceled_items")

+-----------+---------------+
|    country|cancelled_items|
+-----------+---------------+
|New Zealand|              2|
+-----------+---------------+

